# Data collection and setup

*The first step is to create a virtual environment for this project. This is not shown here but it is done anyway and is a good practice.

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlalchemy as sa

## Collect the data

#### Open the database with SQLAlchemy

In [3]:
con = sa.create_engine("sqlite:///" + "../data/hipermercado.db")

In [4]:
from sqlalchemy import inspect
insp = inspect(con)
tables = insp.get_table_names()
tables

['calendar', 'sales', 'sell_prices']

#### There are 3 tables. Let's see them one by one.

In [6]:
cal = pd.read_sql("calendar", con=con)
cal.drop(columns="index", inplace=True)
cal.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2
0,2013-01-01,11249,Tuesday,4,1,2013,d_704,NewYear,National,None,None
1,2013-01-02,11249,Wednesday,5,1,2013,d_705,None,None,None,None
2,2013-01-03,11249,Thursday,6,1,2013,d_706,None,None,None,None
3,2013-01-04,11249,Friday,7,1,2013,d_707,None,None,None,None
4,2013-01-05,11250,Saturday,1,1,2013,d_708,None,None,None,None


In [7]:
sales = pd.read_sql("sales", con=con)
sales.drop(columns="index", inplace=True)
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_704,d_705,d_706,d_707,...,d_1789,d_1790,d_1791,d_1792,d_1793,d_1794,d_1795,d_1796,d_1797,d_1798
0,FOODS_3_090_CA_3_validation,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,0,224,241,232,...,5,2,0,0,6,0,6,0,0,0
1,FOODS_3_120_CA_3_validation,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,54,63,44,0,65,90,104,73,76,97
2,FOODS_3_202_CA_3_validation,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,20,23,23,25,...,43,40,39,0,29,33,27,13,26,47
3,FOODS_3_252_CA_3_validation,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,34,27,40,49,...,31,43,32,0,52,37,32,29,34,27
4,FOODS_3_288_CA_3_validation,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,0,0,0,0,...,29,45,28,0,46,36,40,31,46,36


In [8]:
prices = pd.read_sql("sell_prices", con=con)
prices.drop(columns="index", inplace=True)
prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_3,FOODS_3_090,11249,1.25
1,CA_3,FOODS_3_090,11250,1.25
2,CA_3,FOODS_3_090,11251,1.25
3,CA_3,FOODS_3_090,11252,1.25
4,CA_3,FOODS_3_090,11301,1.38


#### The final dataframe is gonna be a union between all of them. In order to do that, we are going to merge first sales and calendar by a common field which is the day "d". It is necessary to do some arrangements on sales dataset first.

In [10]:
sales = sales.melt(id_vars=sales.columns[0:6],
           var_name="d",
           value_name="sales")

sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales
0,FOODS_3_090_CA_3_validation,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0
1,FOODS_3_120_CA_3_validation,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,d_704,0
2,FOODS_3_202_CA_3_validation,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,d_704,20
3,FOODS_3_252_CA_3_validation,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,d_704,34
4,FOODS_3_288_CA_3_validation,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,d_704,0


In [11]:
# The id column is not needed
sales.drop(columns="id", inplace=True)

In [13]:
# Sales and calendar dataframe generation
df = pd.merge(left=sales, right=cal, on="d", how="left")
df.head()

,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2
0,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
1,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
2,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,d_704,20,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
3,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,d_704,34,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None
4,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None


In [14]:
# Add the prices dataset
df = pd.merge(left=df, right=prices, on=["store_id","item_id","wm_yr_wk"], how="left")
df

,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,sell_price
0,FOODS_3_090,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,1.25
1,FOODS_3_120,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,4.98
2,FOODS_3_202,FOODS_3,FOODS,CA_3,CA,d_704,20,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,4.28
3,FOODS_3_252,FOODS_3,FOODS,CA_3,CA,d_704,34,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,1.48
4,FOODS_3_288,FOODS_3,FOODS,CA_3,CA,d_704,0,2013-01-01,11249,Tuesday,4,1,2013,NewYear,National,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21895,FOODS_3_329,FOODS_3,FOODS,CA_4,CA,d_1798,6,2015-12-31,11548,Thursday,6,12,2015,None,None,None,None,2.98
21896,FOODS_3_555,FOODS_3,FOODS,CA_4,CA,d_1798,5,2015-12-31,11548,Thursday,6,12,2015,None,None,None,None,1.68
21897,FOODS_3_586,FOODS_3,FOODS,CA_4,CA,d_1798,11,2015-12-31,11548,Thursday,6,12,2015,None,None,None,None,1.68
21898,FOODS_3_587,FOODS_3,FOODS,CA_4,CA,d_1798,16,2015-12-31,11548,Thursday,6,12,2015,None,None,None,None,2.48


In [18]:
# Reorder the columns and add the date as index
order = ["date",
        "state_id",
        "store_id",
        "dept_id",
        "cat_id",
        "item_id",
        "wm_yr_wk",
        "d",
        "sales",
        "sell_price",
        "year",
        "month",
        "wday",
        "weekday",
        "event_name_1",
        "event_type_1",
        "event_name_2",
        "event_type_2"]


In [19]:
df = df[order].set_index("date")
df.head()

,state_id,store_id,dept_id,cat_id,item_id,wm_yr_wk,d,sales,sell_price,year,month,wday,weekday,event_name_1,event_type_1,event_name_2,event_type_2
date,,,,,,,,,,,,,,,,,
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_090,11249,d_704,0,1.25,2013,1,4,Tuesday,NewYear,National,None,None
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_120,11249,d_704,0,4.98,2013,1,4,Tuesday,NewYear,National,None,None
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_202,11249,d_704,20,4.28,2013,1,4,Tuesday,NewYear,National,None,None
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_252,11249,d_704,34,1.48,2013,1,4,Tuesday,NewYear,National,None,None
2013-01-01,CA,CA_3,FOODS_3,FOODS,FOODS_3_288,11249,d_704,0,NaN,2013,1,4,Tuesday,NewYear,National,None,None


## Create validation dataset

In [20]:
# The last 30 days of the dataset are going to be used for validation

val = df.loc["2015-12-01":"2015-12-31"]
df = df.loc[:"2015-11-31"]

In [21]:
# Save the validation dataset in a .csv format
val.to_csv("../data/validation.csv")

## Save the working dataset

In [22]:
df.to_csv("../data/work.csv")